<a href="https://colab.research.google.com/github/LeoDardanello/DA_Semantic_Segmentation/blob/main/Semantic_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import sys, os
import numpy as np
import torchvision

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
if os.path.exists("/content/DA_Semantic_Segmentation"):
  !rm -rf "/content/DA_Semantic_Segmentation"
!git clone "https://github.com/LeoDardanello/DA_Semantic_Segmentation.git"

Cloning into 'DA_Semantic_Segmentation'...
remote: Enumerating objects: 5062, done.
remote: Counting objects: 100% (1620/1620), done.
remote: Compressing objects: 100% (1609/1609), done.
remote: Total 5062 (delta 20), reused 1603 (delta 11), pack-reused 3442
Receiving objects: 100% (5062/5062), 3.55 GiB | 15.91 MiB/s, done.
Resolving deltas: 100% (199/199), done.


In [6]:
!7z x '/content/drive/MyDrive/AML/Cityscapes.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/AML/                                     1 file, 4973983436 bytes (4744 MiB)

Extracting archive: /content/drive/MyDrive/AML/Cityscapes.zip
  4% 256 Open              40% 2560 Open               65% 4096 Open               69% 4352 Open               73% 4608 Open               77% 4864 Open               81% 5120 Open               86% 5376 Open               90% 5632 Open               94% 5888 Open

**TRAINING**

In [ ]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --num_workers 2 --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

# GTA 5 TEST


In [ ]:
!7z x '/content/drive/MyDrive/AML/GTA5.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/AML/                                     1 file, 6215145033 bytes (5928 MiB)

Extracting archive: /content/drive/MyDrive/AML/GTA5.zip
  5% 256 Open              10% 512 Open              15% 768 Open              20% 1024 Open               25% 1280 Open               30% 1536 Open               35% 1792 Open               40% 2048 Open               46% 2304 Open               51% 2560 Open        

In [ ]:
# our folder with our labels for GTA5
!unzip -qq '/content/drive/MyDrive/AMLUtils/TrainID.zip' -d '/content/GTA5/TrainID'

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --num_workers 2 --dataset_train "GTA5" --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

In [ ]:
!rm -rf "/content/GTA5/TrainID"

# GTA TRAINING WITH CITYSCAPE TESTING

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --batch_size 8 --learning_rate 0.001 --num_workers 2 --dataset_train "GTA5" --dataset_test "Cityscapes" --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_7391.tar"

# DOMAIN ADAPTATION

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train_with_DA.py" --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

# FOURIER DOMAIN ADAPTATION

In [ ]:
beta=0.05

In [ ]:
# get FDA images
!rm -rf "/content/GTA5/FDA"
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_{beta}/FDA.zip' -d '/content/GTA5/FDA'

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train_with_DA.py" --optimizer "sgd" --num_epochs 1 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar" --lamb 0.001  --checkpoint_step 1 --enable_FDA True --beta {beta}

Generating LabelID..
100% 2500/2500 [00:00<00:00, 252158.52it/s]
Recovering split for beta :  0.05
Generating FDA images..
1572it [00:00, 258181.76it/s]
use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
epoch 0, lr 0.001000: 100% 1568/1568 [05:52<00:00,  4.45it/s]
start val!
100% 500/500 [02:20<00:00,  3.55it/s]
precision per pixel for test: 0.543
mIoU for validation: 0.108
mIoU per class: [5.80324591e-01 9.36138179e-03 5.77852417e-01 8.97833130e-05
 0.00000000e+00 1.91740794e-03 0.00000000e+00 0.00000000e+00
 2.10219285e-01 3.30803770e-04 6.63772851e-01 0.00000000e+00
 0.00000000e+00 1.18570464e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]


In [ ]:
import shutil
shutil.make_archive("/content/FDA", 'zip', "/content/GTA5/FDA")
# shutil.move("/content/FDA.zip", "/content/FDA_005")
# shutil.move("/content/source_data.txt", "/content/FDA_005")
# shutil.move("/content/source_label.txt", "/content/FDA_005")
# shutil.move("/content/FDA_005", "/content/drive/MyDrive/AMLUtils/")

## **Evaluation using all three betas**

In [5]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_run_beta_0.01.zip' -d '/content/FDA_run_beta0.01'

In [4]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_run_beta_0.05.zip' -d '/content/FDA_run_beta0.05'

In [3]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_run_beta_0.09.zip' -d '/content/FDA_run_beta0.09'

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/evaluation_multibeta.py" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
training model with 
start val!
100% 500/500 [02:18<00:00,  3.61it/s]
precision per pixel for test: 0.693
mIoU for validation: 0.252
mIoU per class: [7.82542106e-01 2.48661802e-01 7.11005696e-01 1.20196664e-01
 5.21337228e-02 2.05787615e-01 7.70069240e-04 0.00000000e+00
 7.17159727e-01 1.33393126e-01 6.76010562e-01 2.92797485e-05
 0.00000000e+00 7.29496886e-01 1.72102647e-01 1.33735877e-01
 9.69602458e-02 0.00000000e+00 0.00000000e+00]



**Generating Pseudo Label**



In [ ]:
!python3 "/content/DA_Semantic_Segmentation/evaluation_multibeta.py" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar" --use "generatepseudo"

use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
training model with 
 72% 1139/1572 [03:18<01:10,  6.14it/s]^C


**Finetuning**

In [7]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/pseudolbl.zip' -d '/content/Cityscapes/Cityspaces/pseudolbl'